In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transform
import torch.optim as optim

In [17]:
class Descriminator(nn.Module):
    def __init__(self,imgsize) :
        super().__init__()
        self.dis=nn.Sequential(
            nn.Linear(imgsize,64),
            nn.LeakyReLU(0.1),
            nn.Linear(64,1),
            nn.Sigmoid()
        )
    def forward(self,x):
        return self.dis(x)

In [18]:
class Generator(nn.Module):
    def __init__(self,imgsize,gensize):
        super().__init__()
        self.gen=nn.Sequential(nn.Linear(gensize,64),
        nn.LeakyReLU(0.1),
        nn.Linear(64,imgsize),
        nn.Tanh()
        )
    def forward(self,x):
        return self.gen(x)

In [21]:
imgsize=28*28*1
gensize=32
device="cuda" if torch.cuda.is_available() else "cpu"
lr=3e-4
batch_size=20
epochs=20
disc=Descriminator(imgsize).to(device)
disc.apply()
gene=Generator(imgsize,gensize).to(device)
trans=transform.Compose(
    [transform.ToTensor(),transform.Normalize((0.1307,),(0.3081,))]
)
dataset=datasets.MNIST(root='dataset/',transform=trans,download=True)


In [24]:
data=DataLoader(dataset,batch_size=batch_size,shuffle=True)
disccompile=optim.Adam(disc.parameters(),lr=lr)
gencompile=optim.Adam(gene.parameters(),lr=lr)
lossfunction=nn.BCELoss()
